# CNN Exercise

## 1. Library Call & Setting

In [1]:
from tqdm.notebook import tqdm
import numpy as np
import random
import warnings
warnings.filterwarnings('ignore')

import torch
from torchvision import datasets, transforms

GPU

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

Randomness Control

In [3]:
seed = 42
if device == 'cuda':
    torch.cuda.manual_seed_all(seed)
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

## 2. Data Load

In [4]:
train = datasets.MNIST('./mnist_data/',
                       train=True,
                       transform=transforms.ToTensor(),
                       download=True)

test = datasets.MNIST('./mnist_data/',
                      train=False,
                      transform=transforms.ToTensor(),
                      download=True)

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting ./mnist_data/MNIST/raw/train-images-idx3-ubyte.gz to ./mnist_data/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting ./mnist_data/MNIST/raw/train-labels-idx1-ubyte.gz to ./mnist_data/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting ./mnist_data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./mnist_data/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting ./mnist_data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./mnist_data/MNIST/raw



In [5]:
train

Dataset MNIST
    Number of datapoints: 60000
    Root location: ./mnist_data/
    Split: Train
    StandardTransform
Transform: ToTensor()

In [6]:
test

Dataset MNIST
    Number of datapoints: 10000
    Root location: ./mnist_data/
    Split: Test
    StandardTransform
Transform: ToTensor()

## 3. Model

Model Definition

In [7]:
class CNN(torch.nn.Module):

    def __init__(self):
        super(CNN, self).__init__()

        # input_layer
        # batch_size x  1 x 28 x 28

        # hidden_layer_1
        # batch_size x  1 x 28 x 28 -> batch_size x 32 x 14 x 14 
        self.layer1 = torch.nn.Sequential(
            # batch_size x  1 x 28 x 28 -> batch_size x 32 x 28 x 28
            torch.nn.Conv2d(in_channels= 1, out_channels=32, kernel_size=3, stride=1, padding=1), # Convolution 1: (32,  1, 3, 3), stride=1, padding=1
            torch.nn.ReLU(),
            # batch_size x 32 x 28 x 28 -> batch_size x 32 x 14 x 14 
            torch.nn.MaxPool2d(kernel_size=2, stride=2)) # MaxPooling 1: (2, 2), stride=2, padding=0
            
        # hidden_layer_2
        # batch_size x 32 x 14 x 14 -> batch_size x 64 x  7 x  7   
        self.layer2 = torch.nn.Sequential(
            # batch_size x 32 x 14 x 14 -> batch_size x 64 x 14 x 14 
            torch.nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1), # Convolution 2: (64, 32, 3, 3), stride=1, padding=1
            torch.nn.ReLU(),
            # batch_size x 64 x 14 x 14 -> batch_size x 64 x 7 x 7 
            torch.nn.MaxPool2d(kernel_size=2, stride=2)) # MaxPooling 2: (2, 2)

        # fully_connected_layer
        # batch_size x 64 x  7 x  7 -> batch_size x 10        
        self.fc = torch.nn.Linear(64 * 7 * 7, 10, bias=True)
        torch.nn.init.xavier_uniform_(self.fc.weight)

    def forward(self, x):
        x = self.layer1(x)
        x = self.layer2(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x

In [8]:
model = CNN().to(device)

HyperParemeter

In [9]:
batch_size = 100
epochs = 15
learning_rate = 0.001

optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
criterion = torch.nn.CrossEntropyLoss().to(device)

## 4. Training

In [10]:
train_loader = torch.utils.data.DataLoader(dataset=train,
                                           batch_size=batch_size,
                                           shuffle=True,
                                           drop_last=True)

In [11]:
for epoch in (range(epochs)):
    avg_loss = 0

    for X_train, y_train in tqdm(train_loader):
        X_train = X_train.to(device)
        y_train = y_train.to(device)

        optimizer.zero_grad()
        pred = model(X_train)
        loss = criterion(pred, y_train)
        loss.backward()
        optimizer.step()

        avg_loss += loss / len(train_loader)

    print(f"[{epoch+1:>2}/{epochs}] Loss: {avg_loss:.6f}")

  0%|          | 0/600 [00:00<?, ?it/s]

[ 1/15] Loss: 0.240862


  0%|          | 0/600 [00:00<?, ?it/s]

[ 2/15] Loss: 0.059789


  0%|          | 0/600 [00:00<?, ?it/s]

...

[13/15] Loss: 0.007540


  0%|          | 0/600 [00:00<?, ?it/s]

[14/15] Loss: 0.006813


  0%|          | 0/600 [00:00<?, ?it/s]

[15/15] Loss: 0.006153


## 5. Prediction

In [12]:
with torch.no_grad():
    X_test = test.test_data.view(len(test), 1, 28, 28).float().to(device)
    y_test = test.test_labels.to(device)

    pred_ = model(X_test)
    correct = torch.argmax(pred_, 1) == y_test
    accuracy = correct.float().mean()
    print(f"Accuracy: {accuracy.item()*100:.2f}%")

Accuracy: 98.59%
